## Data Web Scrapping

In [ ]:
pwd #!pwd

In [9]:
import requests

# This helps fetch the page.
# If the response contains the data we are looking for, its likely static.
# If it doesn't, the content is dynamic, meaning it may require Selenium and a WebDriver to extract the data.

url = "https://clark.wa.gov/treasurer/tax-foreclosure-auctions"

response = requests.get(url)
print(response.text)

  
  <!DOCTYPE html>
<html lang="en" dir="ltr" prefix="og: https://ogp.me/ns#">
  <head>
    <meta charset="utf-8" />
<meta name="description" content="   " />
<link rel="canonical" href="https://clark.wa.gov/treasurer/tax-foreclosure-auctions" />
<meta property="og:site_name" content="Clark County" />
<meta property="og:url" content="https://clark.wa.gov/treasurer/tax-foreclosure-auctions" />
<meta property="og:title" content="Tax Foreclosure Auctions | Clark County" />
<meta property="og:description" content="   " />
<meta name="MobileOptimized" content="width" />
<meta name="HandheldFriendly" content="true" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<link rel="icon" href="/favicon.png" type="image/png" />

    <title>Tax Foreclosure Auctions | Clark County</title>
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <link rel="stylesheet" media="all" href="/themes/contrib/stable/css/system/components/align.module.css?sl40a9" />
<link r

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Targeted URL
url = "https://clark.wa.gov/treasurer/tax-foreclosure-auctions"

# Fetch the page content
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')

# Finding the relevant data
tables = soup.find_all('tbody')

# Cleaning up data. Converting the table to a pandas DF for easier wrangling.
#df_list = pd.read_html(str(tables))
#df = pd.concat(df_list)

#print(soup)
print(tables)

[<tbody>
<tr>
<td>
<p class="text-align-center">114781012</p>
</td>
<td>
<p class="text-align-center">N/A</p>
</td>
<td>
<p class="text-align-center">NANCY SUBDIV LOT 2 BLK1</p>
</td>
<td>
<p class="text-align-center">$20,100.00</p>
</td>
<td>
<p class="text-align-center">$47,760.00</p>
</td>
</tr>
</tbody>, <tbody>
<tr>
<td>
<p class="text-align-center">266786000</p>
</td>
<td>
<p class="text-align-center">11601 NE 345th St, La Center, WA 98629</p>
</td>
<td>
<p class="text-align-center">#35 &amp; #75 SEC 34 T5N R2EWM 6.95A</p>
</td>
<td>
<p class="text-align-center">$366,334.00</p>
</td>
<td>
<p class="text-align-center">$366,324.00</p>
</td>
</tr>
<tr>
<td>
<p class="text-align-center">126042318</p>
</td>
<td>
<p class="text-align-center">19501 SE 42nd Cir, Camas, WA 98607</p>
</td>
<td>
<p class="text-align-center">WINCHESTER HILLS PHASE 2 LOT 61 SUB 2000</p>
</td>
<td>
<p class="text-align-center">$459,680.00</p>
</td>
<td>
<p class="text-align-center">$411,620.00</p>
</td>
</tr>


In [ ]:
#pip install selenium

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

# Load the HTML source code from the file
with open("/mnt/data/Clark_county_source_code.txt", "r", encoding="utf-8") as file:
    html_content = file.read()

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Create an empty dictionary to store data by year
yearly_data = {}

# Identify all sections that contain "results-of-the-february-XXXX-tax-foreclosure-sale"
# Replace 'div' with the appropriate tag that holds each year's data
sections = soup.find_all("div", id=lambda x: x and "results-of-the-february" in x)

# Loop through each section
for section in sections:
    # Extract the year from the section id
    year = section.get("id").split("-")[-1]
    
    # Find the <tbody> within that section
    tbody = section.find('tbody')

    # Extract rows from the <tbody>
    rows = tbody.find_all('tr')

    # Prepare a list to store the row data
    table_data = []
    
    for row in rows:
        columns = row.find_all('td')
        row_data = [col.get_text(strip=True) for col in columns]
        table_data.append(row_data)
    
    # Store the data in a dictionary by year
    yearly_data[year] = pd.DataFrame(table_data)

# Save each year's data to a separate CSV file
for year, df in yearly_data.items():
    df.to_csv(f'foreclosure_data_{year}.csv', index=False)

print("Data extraction complete. Each year's data is saved to separate CSV files.")


In [19]:
from bs4 import BeautifulSoup
import pandas as pd

# Load the HTML source code from the file
with open("Clark_county_source_code.txt", "r", encoding="utf-8") as file:
    html_content = file.read()

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Create an empty DataFrame to store combined data for all years
all_data = pd.DataFrame()

# Identify all sections that contain "results-of-the-february-XXXX-tax-foreclosure-sale"
sections = soup.find_all("div", id=lambda x: x and "results-of-the-february" in x)

# Check if any sections are found
if not sections:
    print("No sections found with 'results-of-the-february'. Please check the structure.")
else:
    # Loop through each section
    for section in sections:
        # Extract the year from the section id
        year = section.get("id").split("-")[-1]

        # Print the section to check if it's valid
        print(f"Processing year: {year}")
        print(section.prettify())  # Debugging step to print section

        # Find the <tbody> within that section
        tbody = section.find('tbody')

        if tbody:
            # Extract rows from the <tbody>
            rows = tbody.find_all('tr')

            # Prepare a list to store the row data
            table_data = []
            
            for row in rows:
                columns = row.find_all('td')
                row_data = [col.get_text(strip=True) for col in columns]
                row_data.append(year)  # Add year to the row data
                table_data.append(row_data)
            
            # Create a DataFrame for the current year's data
            df = pd.DataFrame(table_data)
            
            # Append the current year's data to the combined DataFrame
            all_data = pd.concat([all_data, df])
        else:
            print(f"No <tbody> found for section with year {year}.")

    # Reset the index and save the combined data to a single CSV file
    all_data.reset_index(drop=True, inplace=True)
    all_data.to_csv('combined_foreclosure_data.csv', index=False)

    print("Data extraction complete. All data has been saved to 'combined_foreclosure_data.csv'.")


Processing year: sale
<div class="card" id="results-of-the-february-2024-tax-foreclosure-sale">
 <div class="card-header" id="heading_58921293">
  <h5 class="mb-0">
   <button aria-controls="collapse_58921293" aria-expanded="true" class="btn btn-link collapsed" data-target="#collapse_58921293" data-toggle="collapse" type="button">
    <i class="material-icons">
     add
    </i>
    Results of the February 2024 Tax Foreclosure Sale
   </button>
  </h5>
 </div>
 <div aria-labelledby="heading_58921293" class="collapse" data-parent="#accordion_58921293" id="collapse_58921293">
  <div class="card-body">
   <a class="link-anchor float-right d-none" href="/treasurer/tax-foreclosure-auctions#results-of-the-february-2024-tax-foreclosure-sale">
    <i class="material-icons">
     tag
    </i>
   </a>
   <p>
    There were no properties auctioned for sale in February 2024.
   </p>
  </div>
 </div>
</div>

No <tbody> found for section with year sale.
Processing year: sale
<div class="card" id="re